In [1]:
from datasets import load_dataset, Dataset

dataset = load_dataset("IlyaGusev/habr")
dataset = dataset["train"]
dataset

No config specified, defaulting to: habr/default
Found cached dataset habr (/home/kosenko/.cache/huggingface/datasets/IlyaGusev___habr/default/0.0.1/ecfa75bc3a44bd2bb45703a586c49da7792cb834b4ec6dba905ba7a082a448e6)


  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['id', 'language', 'url', 'title', 'text_markdown', 'text_html', 'author', 'original_author', 'original_url', 'lead_html', 'lead_markdown', 'type', 'time_published', 'statistics', 'labels', 'hubs', 'flows', 'tags', 'reading_time', 'format', 'complexity', 'comments'],
    num_rows: 302049
})

In [2]:
dataset[0]

{'id': 12730,
 'language': 'ru',
 'url': 'https://habr.com/ru/post/12730/',
 'title': 'Хочешь в университет — сделай презентацию',
 'text_markdown': 'Абитуриенты, поступающие в Университет Чикаго, уже начиная с этой осени, будут обязаны, помимо сдачи основных экзаменов, представить четырехстраничные презентации в формате PowerPoint — на свободную тему.\nЧастично новые требования связаны с тем, что PowerPoint становится одним из бизнес-инструментов, а кроме того, считают авторы инициативы, в презентации абитуриент сможет раскрыть свои новые качества, которые не смогут проявиться на традиционных экзаменах.\nФормат PowerPoint стал своего рода языком делового общения. Ежедневно, по оценкам Microsoft, в мире показывается порядка 30 млн презентаций.\nvia AP',
 'text_html': '<div xmlns="http://www.w3.org/1999/xhtml">Абитуриенты, поступающие в Университет Чикаго, уже начиная с этой осени, будут обязаны, помимо сдачи основных экзаменов, представить четырехстраничные презентации в формате PowerP

In [3]:
readingCount = []

for item in dataset:
    score = item["statistics"]["readingCount"]
    readingCount.append(score)

In [4]:
dataset = dataset.add_column("readingCount", readingCount)

In [6]:
dataset = dataset.sort("readingCount", reverse=True)

In [9]:
dataset[10]

{'id': 150302,
 'language': 'ru',
 'url': 'https://habr.com/ru/post/150302/',
 'title': 'Учим Python качественно',
 'text_markdown': 'Здравствуйте всем!\nРешил поделиться методом обучения сего мощного, но в одно и тоже время лёгкого языка программирования. Он действительно лёгкий. Вам не надо будет запоминать и вводить лишних символов, которые Вы можете встретить в Си-подобных языках.\nУдобочитаемый синтаксис, прост в обучении, высокоуровневый язык, Объектно-Ориентированый язык программирования (ООП), мощный, интерактивный режим, масса библиотек. Множество иных плюсов... И это всё в одном языке.\nДля начала окунёмся в возможности и узнаем, что же умеет Python?\n#### А зачем мне твой Python?\nМного начинающих программистов задают подобные вопросы. Это как с покупкой телефона, скажите, почему я должен купить этот телефон, а не этот?\n##### Качество программного обеспечения\nДля многих, в том числе и для меня, основные преимущества — это удобочитаемый синтаксис. Не много языков могут похв

In [10]:
dataset = dataset.select(range(10000))

In [11]:
dataset.push_to_hub("dim/habr_10k")

Pushing dataset shards to the dataset hub:   0%|          | 0/2 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

### create prompts


In [2]:
import openai
import pandas as pd
import numpy as np
import time

openai.api_key = open("./chat_gpt_token").read()

In [3]:
def chat_with_chatgpt(prompt, model="gpt-3.5-turbo"):
    chat_completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0,
    )

    return chat_completion["choices"][0]["message"]["content"].strip()


user_prompt = "Hello world."
chatbot_response = chat_with_chatgpt(user_prompt)
print(chatbot_response)

Hello! How can I assist you today?


In [4]:
dataset = load_dataset("dim/habr_10k")
dataset = dataset["train"]

Found cached dataset parquet (/home/kosenko/.cache/huggingface/datasets/dim___parquet/dim--habr_10k-7883d1008129f750/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
dataset

Dataset({
    features: ['id', 'language', 'url', 'title', 'text_markdown', 'text_html', 'author', 'original_author', 'original_url', 'lead_html', 'lead_markdown', 'type', 'time_published', 'statistics', 'labels', 'hubs', 'flows', 'tags', 'reading_time', 'format', 'complexity', 'comments', 'readingCount'],
    num_rows: 10000
})

In [ ]:
dataset[0]

In [5]:
dataset = dataset.select(range(5000))

In [6]:
from tqdm import tqdm
import time
prompts = []

for item in tqdm(dataset):
    title = item["title"]
    # print(title)
    is_generated = False
    while not is_generated:
        try:
            user_prompt = f"перефразируй данный заголовок от статьи так, чтобы он звучал как запрос от пользователя: {title}"
            result = chat_with_chatgpt(user_prompt)
            is_generated = True
            # print(result)
            prompts.append(result)
            time.sleep(3)
            # print("-" * 30)
        except:
            pass
    # break

100%|██████████| 5000/5000 [7:30:31<00:00,  5.41s/it]     


In [8]:
len(prompts)

5000

In [9]:
dataset

Dataset({
    features: ['id', 'language', 'url', 'title', 'text_markdown', 'text_html', 'author', 'original_author', 'original_url', 'lead_html', 'lead_markdown', 'type', 'time_published', 'statistics', 'labels', 'hubs', 'flows', 'tags', 'reading_time', 'format', 'complexity', 'comments', 'readingCount'],
    num_rows: 5000
})

In [11]:
prompts[100]

'Как я могу полностью отключить Защитник Windows на Windows 10?'

In [12]:
dataset = dataset.add_column("prompts", prompts)

In [14]:
dataset[0]

{'id': 491974,
 'language': 'ru',
 'url': 'https://habr.com/ru/post/491974/',
 'title': 'Коронавирус: почему надо действовать прямо сейчас',
 'text_markdown': "## Вступление\nУчитывая всё, что происходит с коронавирусом, может оказаться очень сложно принять решение, что делать прямо сейчас. Стоит ли подождать, пока станет больше информации? Надо ли предпринять что-то уже сегодня? Если да, то что?\nВ этой статье со множеством графиков, данных и моделей из большого числа источников мы постараемся ответить на вопросы:\n   Сколько людей заболеют коронавирусом в вашем регионе?\n   Что случится, когда они начнут заболевать?\n   Что вы должны делать?\n   Когда?\nКогда вы закончите читать статью, вы придёте к следующим выводам:\n   Коронавирус приближается к вам.\n   Он приближается с экспоненциальной скоростью: сперва постепенно, а потом внезапно.\n   Это вопрос нескольких дней. Может быть, неделя или две.\n   Когда это случится, ваша система здравоохранения будет перегружена.\n   Ваши сограж

In [15]:
dataset.push_to_hub("dim/habr_prompts_5k", private=True)

Pushing dataset shards to the dataset hub:   0%|          | 0/2 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]